# The Parameter Tree

**Getting familiar with the PtyPy parameter tree using the MoonFlower simulation/reconstruction.**

In PtyPy, the parameter tree is a Python object - more specifically a dictionary-like [ptypy.utils.Param](https://ptycho.github.io/ptypy/rst/ptypy.utils.html#ptypy.utils.parameters.Param) object. This means that we can put together a Python script (or Jupyter notebook), define all relevant parameters and execute the script (notebook) to run a ptychographic reconstruction! Let's have a look at this parameter tree and its building blocks...

<div class="alert alert-success" markdown="1">
<strong>Tip</strong><br>A full list of available parameters can be found here: <a href="https://ptycho.github.io/ptypy/rst/parameters.html">https://ptycho.github.io/ptypy/rst/parameters.html</a>
</div>

First, we need to import ```ptypy```.
```python
import ptypy
import ptypy.utils as u
```

We start by creating an empty root parameter tree ```p```.
```python
p = u.Param()
```

We set the ```verbose_level``` to ```"interactive"``` which is preferred for Jupyter notebooks since it gives a more condensed user output. You can use ```"info"``` or ```"debug"``` for a more detailed output.
```python
p.verbose_level = "interactive"
```

Next we create a subtree for input/output related parameters. For this particular example, we don't save any files and turn off autoplotting.
```python
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)
```

<div class="alert alert-info" markdown="1">
<strong>Note</strong><br>Parameters can be specified as arguments to the <code>u.Param</code> class when creating a tree/subtree, like
<pre><code>p.io.interaction = u.Param(active=False)
</code></pre>
instead of
<pre><code>p.io.interaction = u.Param()
p.io.interaction.active = False
</code></pre>
</div>

PtyPy allows us to define multiple scans in a single reconstruction, but in most cases we just define a single scan instance. In this case, we create a scan of type (scan model) ```"Full"``` called ```MF``` (named after the **M**oon**F**lower simulation example used here).
```python
p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"
```

The scan model creates the [Views](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.View), [Containers](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.Container) and [Storages](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.Storage) for the ```object```, ```probe``` and other data structures including the [PODs](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.classes.POD) which links them all together. 

<div class="alert alert-success" markdown="1">
<strong>Tip</strong><br>To learn more about these classes and their underlying concepts, visit this <a href url="https://ptycho.github.io/ptypy/rst/concept.html">tutorial</a> from the main PtyPy documentation.
</div>

As part of the scan we also need to define a ```data``` subtree, holding all relevant information related to the data (or simulated data) we are trying to reconstruct. For a real experiment, users can create their own data loader by subclassing [ptypy.core.data.PtyScan](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.data.PtyScan) or using the generic [ptypy.experiment.hdf5_loader.Hdf5Loader](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.data.PtyScan). In this example, we are using the ```MoonFlowerScan``` which provides simulated diffraction data from a synthetic probe (the moon) and a synthetic object (the flower).
```python
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.
```

<div class="alert alert-success" markdown="1">
<strong>Tip</strong><br>To learn more about data management and how to subclass <code>PtyScan</code>, visit this <a href url="https://ptycho.github.io/ptypy/rst/data_management.html">tutorial</a> from the main PtyPy documentation.
</div>

Finally, we need to define one (or more) reconstruction engines which specify which ptychographic algorithm(s) should be used in the reconstruction. In this example, we define ```engine00``` to use the difference map (DM) algorithm with 80 iterations.
```python
p.engines = u.Param()
p.engines.engine00 = u.Param()
p.engines.engine00.name = "DM"
p.engines.engine00.numiter = 80
```

At the end of our script (notebook) we can create a [ptypy.core.Ptycho](https://ptycho.github.io/ptypy/rst/ptypy.core.html#ptypy.core.ptycho.Ptycho) object by passing the parameter tree ```p``` and ```level=5``` which initialises everything and starts the reconstruction.
```python
P = ptypy.core.Ptycho(p,level=5)
```

After loading/simulating the diffraction data, initialising the model/engine and running 80 iterations of difference map the output should look something like this
```bash
Full: loading data for scan MF (161 diffraction frames, 161 PODs, 1 probe(s) and 1 object(s))
Full: loading data for scan MF (reformatting probe/obj/exit)
Full: loading data for scan MF (initializing probe/obj/exit)
DM: initializing engine
DM: preparing engine
DM: Iteration # 80/80 :: Fourier 5.37e+01, Photons 1.55e+01, Exit 4.78e+00
==== This reconstruction relied on the following work ==========================
The Ptypy framework:
    Enders B. and Thibault P., "A computational framework for ptychographic reconstructions" Proc. Royal Soc. A 472 (2016) 20160640, doi: 10.1098/rspa.2016.0640.
The difference map reconstruction algorithm:
    Thibault et al., "Probe retrieval in ptychographic coherent diffractive imaging" Ultramicroscopy 109 (2009) 338, doi: 10.1016/j.ultramic.2008.12.011.
================================================================================
```

To visualise the results of the reconstruction, we can use ```u.plot_recon_from_ptycho(P)``` to produce an image like this 

![](_assets/moonflower_recons_plot.png)

<div class="alert alert-warning" markdown="1">
    <strong>Exersice</strong><br>Run the MoonFlower example below and inspect the attributes of the <code>Ptycho</code> instance <code>P</code>. Can you figure out what the reconstructed pixel size is for this example?
</div>

<div class="alert alert-warning" markdown="1">
    <strong>Exersice</strong><br>Change the <code>verbose_level</code> to <code>info</code> or <code>debug</code> and observe the change in output. Can you find a reference to the reconstructed pixel size in the log?
</div>

<div class="alert alert-warning" markdown="1">
    <strong>Exersice</strong><br>Where the <code>Ptycho</code> instance is being created, change the <code>level</code> from 5 to 4 and observe the output. Did PtyPy actually run the engine? and what happens if we now do <code>P.run()</code>?
</div>

---

In [ ]:
import ptypy
import ptypy.utils as u

# Create parameter tree
p = u.Param()

# Set verbose level, can be "interactive", "info" or "debug"
p.verbose_level = "interactive"

# Basic I/O settings (no files saved in this case)
p.io = u.Param()
p.io.rfile = None
p.io.autosave = u.Param(active=False)
p.io.autoplot = u.Param(active=False)
p.io.interaction = u.Param(active=False)

# Define the scan model
p.scans = u.Param()
p.scans.MF = u.Param()
p.scans.MF.name = "Full"

# Data loader / simulator 
# Generate 200 frames (128x128px) of diffraction data
p.scans.MF.data= u.Param()
p.scans.MF.data.name = "MoonFlowerScan"
p.scans.MF.data.shape = 128
p.scans.MF.data.num_frames = 200
p.scans.MF.data.save = None
p.scans.MF.data.density = 0.2
p.scans.MF.data.photons = 1e8
p.scans.MF.data.psf = 0.

# Define reconstruction engine
p.engines = u.Param()
p.engines.engine00 = u.Param()
p.engines.engine00.name = "DM"
p.engines.engine00.numiter = 80

# Prepare and run
P = ptypy.core.Ptycho(p,level=5)

# Display the results
u.plot_recon_from_ptycho(P)